In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv('insurance.csv', sep=',')
data_train = data.sample(frac=0.8)
data_test = data.loc[~data.index.isin(data_train.index)]

**ADD COLLUMN TRANSFORMER TO SCALE AND TRANSFORM DESCRETE COLUMNS; ADD PIPELINE**

In [3]:
col_trans = ColumnTransformer([
    ('st_scller', StandardScaler(), ('age', 'bmi', 'children')), 
    ('hot_encode', OneHotEncoder(handle_unknown='ignore'), ('sex', 'smoker', 'region'))
], remainder='passthrough')
pipe = Pipeline([('col_trans', col_trans), ('poly_trans', PolynomialFeatures()), ('lasso', Lasso())])

In [4]:
col_trans.fit_transform(data_train)

array([[-1.45188412e+00, -1.03649236e+00, -8.75700024e-02, ...,
         0.00000000e+00,  0.00000000e+00,  2.70911190e+03],
       [ 4.10329628e-01, -1.55550287e+00, -9.09498972e-01, ...,
         0.00000000e+00,  0.00000000e+00,  7.22278625e+03],
       [-6.64024456e-01, -1.28813382e+00,  7.34358967e-01, ...,
         0.00000000e+00,  0.00000000e+00,  1.73617661e+04],
       ...,
       [-1.45188412e+00,  3.87268532e-01, -9.09498972e-01, ...,
         1.00000000e+00,  0.00000000e+00,  3.44398559e+04],
       [-9.50518878e-01,  1.55856019e+00, -9.09498972e-01, ...,
         0.00000000e+00,  0.00000000e+00,  3.20124515e+03],
       [ 1.26981290e+00,  2.14048107e+00, -8.75700024e-02, ...,
         0.00000000e+00,  1.00000000e+00,  1.15761300e+04]])

In [5]:
pipe.fit(data_train.loc[:, ~data_train.columns.isin(['charges'])], data_train['charges'])

D:\Programming\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.872e+08, tolerance: 1.601e+07
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('col_trans',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('st_scller', StandardScaler(),
                                                  ('age', 'bmi', 'children')),
                                                 ('hot_encode',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ('sex', 'smoker',
                                                   'region'))])),
                ('poly_trans', PolynomialFeatures()), ('lasso', Lasso())])

In [6]:
pipe.predict(data_train.loc[:, ~data_train.columns.isin(['charges'])])

array([ 4488.33618237,  6598.24472941, 17587.08014862, ...,
       29715.19214199,  5252.90861158, 12268.36049458])

**APPLY GRID SEARCH TO SELECT HYPERPARAMETERS**

In [10]:
%%time
params = dict(
    lasso__alpha= np.linspace(0.001, 10, 100), 
    poly_trans__degree = [2, 3, ], 
    poly_trans__include_bias = [False, ], 
)
alfa_search = GridSearchCV(pipe, params, scoring='neg_mean_absolute_error', verbose=1, n_jobs=8)
alfa_search.fit(data_train.loc[:, ~data_train.columns.isin(['charges'])], data_train['charges'])

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Wall time: 19.8 s


GridSearchCV(estimator=Pipeline(steps=[('col_trans',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('st_scller',
                                                                         StandardScaler(),
                                                                         ('age',
                                                                          'bmi',
                                                                          'children')),
                                                                        ('hot_encode',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ('sex',
                                                                          'smoker',
                                                                          'region'))])),
                                       ('poly_trans', PolynomialFeatures()),
                                       ('lasso', Lasso())]),
             n_jobs=8,
             param_grid={'lasso__alpha': array([1.00...
       7.273e+00, 7.374e+00, 7.475e+00, 7.576e+00, 7.677e+00, 7.778e+00,
       7.879e+00, 7.980e+00, 8.081e+00, 8.182e+00, 8.283e+00, 8.384e+00,
       8.485e+00, 8.586e+00, 8.687e+00, 8.788e+00, 8.889e+00, 8.990e+00,
       9.091e+00, 9.192e+00, 9.293e+00, 9.394e+00, 9.495e+00, 9.596e+00,
       9.697e+00, 9.798e+00, 9.899e+00, 1.000e+01]),
                         'poly_trans__degree': [2, 3],
                         'poly_trans__include_bias': [False]},
             scoring='neg_mean_absolute_error', verbose=1)

In [8]:
alfa_search.best_score_

-3076.9422029396883

In [9]:
alfa_search.best_score_

-3076.9422029396883

# **ДЗ**

**1. Сделать это только предварительно выбрать валидационный базис**
**2. Сделать то же самое для дерева (пайп и grid search)**
**3. Из матрицы результатов, получаемых при разных гиперпараметрах, посмотреть какие наиболее значимые**
**4. Сделать то же для случайного леса (пайп и grid search)**